This exercise will require you to pull some data from the Quandl API. Quandl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [3]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY =''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [4]:
# First, import the relevant modules
import requests
import json

In [5]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "".join(("https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=", API_KEY))
r = requests.get(url)

In [6]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
# print(r.json()['dataset_data']['column_names'])

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### 1.) Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [7]:
afx_x = list()
for i in range(len(r.json()['dataset_data']['data'])):
    if r.json()['dataset_data']['data'][i][0][0:4] == '2017':
        afx_x.append(r.json()['dataset_data']['data'][i])              

### 2.) Convert the returned JSON object into a Python dictionary.

In [8]:
stockdata = dict()
for i in range(len(r.json()['dataset_data']['data'])):
    if r.json()['dataset_data']['data'][i][0][0:4] == '2017':
        date = r.json()['dataset_data']['data'][i][0]
        stockdata[date] = dict()
        stockdata[date]['Open'] = r.json()['dataset_data']['data'][i][1]
        stockdata[date]['High'] = r.json()['dataset_data']['data'][i][2]
        stockdata[date]['Low'] = r.json()['dataset_data']['data'][i][3]
        stockdata[date]['Close'] = r.json()['dataset_data']['data'][i][4]
        stockdata[date]['Change'] = r.json()['dataset_data']['data'][i][5]
        stockdata[date]['Trading Volume'] = r.json()['dataset_data']['data'][i][6]
        stockdata[date]['Turnover'] = r.json()['dataset_data']['data'][i][7]
        stockdata[date]['Last Price of the Day'] = r.json()['dataset_data']['data'][i][8]
        stockdata[date]['Daily Traded Units'] = r.json()['dataset_data']['data'][i][9]
        stockdata[date]['Daily Turnover'] = r.json()['dataset_data']['data'][i][10]

### 3.) Calculate what the highest and lowest opening prices were for the stock in this period.

In [9]:
# Is there a cleaner way to do this?
periodhigh = 0
periodlow = 1e99
for key in stockdata:
    if(stockdata[key]['High'] > periodhigh):
        periodhigh = stockdata[key]['High']
    if(stockdata[key]['Low'] < periodlow):
        periodlow = stockdata[key]['Low']

In [10]:
print("High:", periodhigh,'\nLow:', periodlow)

High: 53.54 
Low: 33.62


### 4.) What was the largest change in any one day (based on High and Low price)?

In [11]:
largestchange = 0
for key in stockdata:
    dailychange = stockdata[key]['High'] - stockdata[key]['Low']
    if dailychange > largestchange:
        largestchange = dailychange
print(round(largestchange, 2))        

2.81


### 5.) What was the largest change between any two days (based on Closing Price)?

In [23]:
closelist = []
maxclose = -1e99
for key in stockdata:
    closelist.append(stockdata[key]['Close'])
i = 0
for i in range(len(closelist)):
    if i > 0:
        if closelist[i] - closelist[i-1] > maxclose:
            maxclose = closelist[i] - closelist[i - 1]
print(round(maxclose, 2))            

2.56


### 6.) What was the average daily trading volume during this year?

In [13]:
totalvolume = 0
totaldays = 0
for key in stockdata:
    totalvolume += stockdata[key]['Trading Volume']
    totaldays += 1
print(round(totalvolume / totaldays, 0))    

89124.0


### 7.) (Optional) What was the median trading volume during this year? (Note: you may need to implement your own function for calculating the median.)

In [14]:
dailyvolume = []
for key in stockdata:
    dailyvolume.append(stockdata[key]['Trading Volume'])
dailyvolume.sort()
median = dailyvolume[int(round(len(dailyvolume) / 2))]
print(median)

76600.0
